In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
import matplotlib
matplotlib.use("Agg")
from matplotlib import pyplot as plt
import imutils
from imutils import paths
import numpy as np
import random
import pickle
import cv2
import os

In [ ]:
def build(width, height, depth, classes):
	model = Sequential()
	inputShape = (height, width, depth)
	chanDim = -1
	if K.image_data_format() == "channels_first":
		inputShape = (depth, height, width)
		chanDim = 1
            
	model.add(Conv2D(32, (3, 3), padding="same",
		input_shape=inputShape))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=chanDim))
	model.add(MaxPooling2D(pool_size=(3, 3)))
	model.add(Dropout(0.25))
        
	model.add(Conv2D(64, (3, 3), padding="same"))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=chanDim))
	model.add(Conv2D(64, (3, 3), padding="same"))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=chanDim))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Dropout(0.25))
        
	model.add(Conv2D(128, (3, 3), padding="same"))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=chanDim))
	model.add(Conv2D(128, (3, 3), padding="same"))
	model.add(Activation("relu"))
	model.add(BatchNormalization(axis=chanDim))
	model.add(MaxPooling2D(pool_size=(2, 2)))
	model.add(Dropout(0.25))
        
	model.add(Flatten())
	model.add(Dense(1024))
	model.add(Activation("relu"))
	model.add(BatchNormalization())
	model.add(Dropout(0.5))
        
	model.add(Dense(classes))
	model.add(Activation("softmax"))
	return model

In [2]:
d = "C:/Users/DELL/Desktop/asl_dataset"
p = "C:/Users/DELL/Desktop"
m = "C:/Users/DELL/Desktop/asl_dataset/model.h5"
lab_bin = "C:/Users/DELL/Desktop/asl_dataset/labelbin"

In [ ]:
EPOCHS = 100
INIT_LR = 1e-3
BS = 32
IMAGE_DIMS = (96, 96, 3)

data = []
labels = []
imagePaths = sorted(list(paths.list_images(d)))
random.seed(42)
random.shuffle(imagePaths)

for imagePath in imagePaths:
	image = cv2.imread(imagePath)
	image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
	image = img_to_array(image)
	data.append(image)
	label = imagePath.split(os.path.sep)[-2]
	labels.append(label)

In [ ]:
data = np.array(data) / 255.0
labels = np.array(labels)
print("[INFO] data matrix: {:.2f}MB".format(
	data.nbytes / (1024 * 1000.0)))

lb = LabelBinarizer()
labels = lb.fit_transform(labels)
(trainX, testX, trainY, testY) = train_test_split(data,
	labels, test_size=0.2, random_state=42)
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
	height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
	horizontal_flip=True, fill_mode="nearest")
print("[INFO] compiling model...")

model = build(width=IMAGE_DIMS[1], height=IMAGE_DIMS[0],
	depth=IMAGE_DIMS[2], classes=len(lb.classes_))
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="categorical_crossentropy", optimizer=opt,
	metrics=["accuracy"])
print("[INFO] training network...")

H = model.fit(
	x=aug.flow(trainX, trainY, batch_size=BS),
	validation_data=(testX, testY),
	steps_per_epoch=len(trainX) // BS,
	epochs=EPOCHS, verbose=1)

In [ ]:
print("[INFO] serializing network...")
model.save(m, save_format="h5")
print("[INFO] serializing label binarizer...")
f = open(lab_bin, "wb")
f.write(pickle.dumps(lb))
f.close()
plt.style.use("ggplot")
plt.figure()

N = EPOCHS

plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper left")
plt.savefig(p)

In [3]:
import tkinter as tk
from tkinter import filedialog, Text

In [7]:
def addFile():
    filename = filedialog.askopenfilename(initialdir="C:/Users/DELL/Desktop/asl_dataset",
                                          title="Select file", filetypes=[("image", "*.jpg"),
                                                                          ("image", "*.jpeg"),
                                                                          ("image", "*.png")])
    global i
    i = filename

In [28]:
root = tk.Tk()

canvas = tk.Canvas(root, height=100, width=150, bg="white")
canvas.pack()

frame = tk.Frame(root, bg="white")
frame.place(relwidth=0.8, relheight=0.6, relx=0.1, rely=0.1)

openFile = tk.Button(root, text="Open File", padx=10, pady=5, fg="white", bg="#263D42", command=addFile)
openFile.pack()

root.mainloop()

In [13]:
#i = input("Enter file location of image: ")

image = cv2.imread(i)
output = image.copy()

image = cv2.resize(image, (96, 96))
image = image.astype("float") / 255.0
image = img_to_array(image)
image = np.expand_dims(image, axis=0)

print("[INFO] loading network...")
model = load_model(m)
l = pickle.loads(open(lab_bin, "rb").read())

print("[INFO] classifying image...")
proba = model.predict(image)[0]
idx = np.argmax(proba)
label = l.classes_[idx]
filename = i[i.rfind(os.path.sep) + 1:]
label = "{}: {:.2f}%".format(label, proba[idx] * 100)

output = imutils.resize(output, width=400)
cv2.putText(output, label, (10, 25),  cv2.FONT_HERSHEY_SIMPLEX,
	0.7, (0, 255, 0), 2)

print("[INFO] {}".format(label))
cv2.imshow("Output", output)
cv2.waitKey(0)

[INFO] loading network...
[INFO] classifying image...
[INFO] x: 99.99%


-1